In [169]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LogisticRegression

In [2]:
data = pd.read_excel("https://www.foreignlaborcert.doleta.gov/docs/py2015q4/H-1B_Disclosure_Data_FY15_Q4.xlsx")

In [128]:
def getcsv():
    '''The function downloads the 'H1B data'
    Author: vys217
    '''
    print ("Downloading")
    
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + "H-1B_Disclosure_Data_FY15_Q4.xlsx"):
        if os.path.isfile("H-1B_Disclosure_Data_FY15_Q4.xlsx"):
            # if in the current dir just move it
            if os.system("mv " + "H-1B_Disclosure_Data_FY15_Q4.xlsx " + os.getenv("PUIDATA")):
                print ("Error moving file!, Please check!")
        #otherwise start looking for the zip file
        else:
            if not os.path.isfile(os.getenv("PUIDATA") + "/" + "H-1B_Disclosure_Data_FY15_Q4.xlsx"):
                if not os.path.isfile("H-1B_Disclosure_Data_FY15_Q4.xlsx"):
                    os.system("wget https://www.foreignlaborcert.doleta.gov/docs/py2015q4/H-1B_Disclosure_Data_FY15_Q4.xlsx")
                    
                ###  To move it I use the os.system() functions to run bash commands with arguments
                os.system("mv " + "H-1B_Disclosure_Data_FY15_Q4.xlsx " + os.getenv("PUIDATA"))

    ### One final check:
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + "H-1B_Disclosure_Data_FY15_Q4.xlsx"):
        print ("WARNING!!! something is wrong: the file is not there!")

    else:
        print ("file in place, you can continue")
        
    print ("Downloading")
    
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + "2017_NAICS_Structure.xlsx"):
        if os.path.isfile("2017_NAICS_Structure.xlsx"):
            # if in the current dir just move it
            if os.system("mv " + "2017_NAICS_Structure.xlsx " + os.getenv("PUIDATA")):
                print ("Error moving file!, Please check!")
        #otherwise start looking for the zip file
        else:
            if not os.path.isfile(os.getenv("PUIDATA") + "/" + "2017_NAICS_Structure.xlsx"):
                if not os.path.isfile("2017_NAICS_Structure.xlsx"):
                    os.system("wget http://www.census.gov/eos/www/naics/2017NAICS/2017_NAICS_Structure.xlsx")
                    
                ###  To move it I use the os.system() functions to run bash commands with arguments
                os.system("mv " + "2017_NAICS_Structure.xlsx " + os.getenv("PUIDATA"))

    ### One final check:
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + "2017_NAICS_Structure.xlsx"):
        print ("WARNING!!! something is wrong: the file is not there!")

    else:
        print ("file in place, you can continue")    

In [129]:
getcsv()

Downloading
file in place, you can continue
Downloading
file in place, you can continue


In [147]:
data = pd.read_excel(os.getenv('PUIDATA')+'/'+"H-1B_Disclosure_Data_FY15_Q4.xlsx")

In [149]:
codes = pd.read_excel( os.getenv('PUIDATA')+'/'+"2017_NAICS_Structure.xlsx", header= 2)
codes.drop(['Change Indicator'], axis= 1, inplace= True)
codes.rename(columns={'2017 NAICS Code' : 'NAIC_CODE'}, inplace=True)

In [16]:
data['decision_date'] = data.DECISION_DATE - data.CASE_SUBMITTED
data = data[data.VISA_CLASS == 'H-1B']

In [84]:
data1 = data.copy()

In [88]:
data1.drop(['EMPLOYER_ADDRESS1', 'EMPLOYER_ADDRESS2', 'EMPLOYER_CITY', 'EMPLOYER_STATE', 
                     'EMPLOYER_POSTAL_CODE', 'VISA_CLASS', 'EMPLOYER_COUNTRY', 'EMPLOYER_PROVINCE',
                     'EMPLOYER_PHONE', 'EMPLOYER_PHONE_EXT', 'AGENT_ATTORNEY_NAME', 'AGENT_ATTORNEY_CITY',
                     'AGENT_ATTORNEY_STATE', 'SOC_CODE', 'PW_WAGE_SOURCE', 'PW_WAGE_SOURCE_YEAR', 
                     'PW_WAGE_SOURCE_OTHER', 'WORKSITE_COUNTY'], axis = 1, inplace = True)

In [151]:
data1 = data1[(dataUS == 'CERTIFIED') | (data1.CASE_STATUS == 'DENIED')]
data2 = data1.dropna(subset=['W1.CASE_STATAGE_UNIT_OF_PAY'])

In [152]:
def convertwage(wage, unit, FT):
   if unit == 'Year':
       return wage
   elif unit == 'Month':
       return wage * 12
   elif unit == 'Week':
       return wage * 4 *12
   elif unit == 'Bi-Weekly':
       return wage * 2 * 12
   elif unit == 'Hour':
       if FT == 'Y':
           return wage * 40 * 4 * 12
       elif FT == 'N':
           return wage * 20 * 4 * 12
Convertwage = np.vectorize(convertwage)

In [153]:
def rangeconvert(w_str):
   w_list = np.array(w_str.split("-"))
   try:
       return (w_list[0].astype(float) + w_list[1].astype(float)) / 2
   except ValueError:
       return w_list[0].astype(float)
Rangeconvert = np.vectorize(rangeconvert)

In [154]:
data2['WAGE_annual'] = Convertwage(Rangeconvert(data2['WAGE_RATE_OF_PAY']), 
                                  data2['WAGE_UNIT_OF_PAY'], data2['FULL_TIME_POSITION'])

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [156]:
data2['PW_WAGE_annual'] = Convertwage(data2['PREVAILING_WAGE'], data2['PW_UNIT_OF_PAY'], data2['FULL_TIME_POSITION'])

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [159]:
data3 = pd.merge(data2, codes, on='NAIC_CODE')